In [102]:
import pandas as pd
import numpy as np

In [103]:
df = pd.read_csv("31_Loc_state/ECN_2012_US_31SA1_with_ann.csv")

In [104]:
print(df.iloc[0:4,])

                       GEO.id GEO.id2     GEO.display-label         NAICS.id  \
0  Geographic identifier code     Id2  Geographic area name  2012 NAICS code   
1                 0400000US01      01               Alabama            31-33   
2                 0400000US01      01               Alabama            31-33   
3                 0400000US01      01               Alabama            31-33   

          NAICS.display-label                         EMPSZES.id  \
0  Meaning of 2012 NAICS code  Employment size of establishments   
1               Manufacturing                                001   
2               Manufacturing                                219   
3               Manufacturing                                220   

                          EMPSZES.display-label YEAR.id  \
0  Meaning of Employment size of establishments    Year   
1                            All establishments    2012   
2          Establishments with 0 to 4 employees    2012   
3          Establishme

In [105]:
#the spreadsheet contains numbers of "establishments" (businesses)
# in the column ESTAB
#these are grouped by: 
# state (GEO.id2 or GEO.display-label)
# area  (NAICS.id or NAICS.display-label)
# number of employees (EMPSZES.id or EMPSZES.display-label)

In [106]:
#the spreadsheet has a weird structure where the header gives column names,
# and then the second row gives additional information, and the third and following are data...
#need to get rid of the second row
#also get rid of some redundant columns


In [107]:
df = df.iloc[1:,]
df = df.drop("GEO.id", axis=1)
df = df.drop("GEO.id2", axis=1)
df = df.drop("NAICS.id",axis=1)
df = df.drop("YEAR.id", axis=1)
df = df.drop("EMPSZES.display-label", axis=1)
df = df.rename(columns={"GEO.display-label":"State",\
                        "NAICS.display-label":"NAICS_Sector",\
                        "EMPSZES.id":"Size_Code",\
                        "ESTAB":"N_Establishments"})

In [108]:
print(df.iloc[0:4,])

     State   NAICS_Sector Size_Code N_Establishments
1  Alabama  Manufacturing       001             4286
2  Alabama  Manufacturing       219             1695
3  Alabama  Manufacturing       220              534
4  Alabama  Manufacturing       230              559


In [109]:


#now estimate number of employees
#first map size code to mean number of employees
#note that size code is string representation of number, keep as string

    
get_mean = lambda low,high : (high*(high + 1)/2.0 - (low-1)*low/2.0)/(high-low + 1)
    
size_code_to_mean_employee_number ={\
            "001":0,\
            "219":get_mean(0,4),\
            "220":get_mean(5,9),\
            "230":get_mean(10,19),\
            "241":get_mean(20,49),\
            "242":get_mean(50,99),\
            "251":get_mean(100,249),\
            "252":get_mean(250,499),\
            "254":get_mean(500,999),\
            "261":get_mean(1000,2499),\
            "270":2500
            }
    


In [110]:
df["N_Employees"] = df["N_Establishments"].astype("float")*df["Size_Code"].map(size_code_to_mean_employee_number)


df.loc[np.logical_not(df["Size_Code"]=="001"),"N_Establishments"] = 0
#set things up so that the groupby sum will give the correct number of establishments
#by getting rid of all counts except for total count

In [111]:
print(df.iloc[0:10,])

      State   NAICS_Sector Size_Code N_Establishments  N_Employees
1   Alabama  Manufacturing       001             4286          0.0
2   Alabama  Manufacturing       219                0       3390.0
3   Alabama  Manufacturing       220                0       3738.0
4   Alabama  Manufacturing       230                0       8105.5
5   Alabama  Manufacturing       241                0      21045.0
6   Alabama  Manufacturing       242                0      26373.0
7   Alabama  Manufacturing       251                0      57236.0
8   Alabama  Manufacturing       252                0      44940.0
9   Alabama  Manufacturing       254                0      48717.5
10  Alabama  Manufacturing       261                0      29741.5


In [118]:
df = df.drop("Size_Code", axis=1)
df["N_Establishments"] = df["N_Establishments"].astype("float")
#set type from object to float, so groupby().sum() will compute sum

In [119]:
#now we just need to sum grouped by state and NAICS sector
#and we will have the total number of establishments
#and the estimated total number of employees

df_sum = df.groupby(by=["State","NAICS_Sector"], axis=0,sort=False).sum().reset_index()

In [120]:
print(df_sum.iloc[0:4,])

     State                     NAICS_Sector  N_Establishments  N_Employees
0  Alabama                    Manufacturing            4286.0     253286.5
1  Alabama               Food manufacturing             263.0      35983.5
2  Alabama   Dog and cat food manufacturing               4.0        525.5
3  Alabama  Other animal food manufacturing              31.0        794.5


In [121]:
df_sum.to_csv("US_Census_2012_business_by_state.csv")